In [1]:
import sys
import os
import math

# Get the current working directory and the relative path to your module
module_path = os.path.abspath(os.path.join("..", ".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from src.api.repository.search import SearchRepo

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
es = SearchRepo()

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '9I9q3mFYSISbvBg75cK23A',
 'name': '5eeda383872f',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


In [3]:
idx = "vehicle"

In [4]:
def number_of_search_terms_from(query_string):
    """
    Returns the number of terms in the query string using space as a separator
    :param query_string: String containing zero or more words separated by spaces
    :return: Number with the amount of found terms
    """
    return len(query_string.split())


In [17]:
def get_completion_prefix(input_string):
    # if the input string is too short, then don't attempt completion
    if len(input_string) < 2:
        return None

    # get the last uncompleted string
    input_string = input_string.lstrip()
    last_space_index = input_string.rfind(" ")
    prefix = input_string[last_space_index + 1 :]

    # if the prefix is 1 or less chars then include the previous word in the prefix
    if len(prefix) <= 1:
        previous_space_index = input_string[:last_space_index].rfind(" ")
        prefix = input_string[previous_space_index + 1 :]

    return prefix


In [58]:
context = ["ABS", "Airbags"]

In [99]:
query_string = "stability co"

In [100]:
query = None
if not context:
    query = None
else:
    query = {
        "bool": {
            "filter": [
                {
                    "nested": {
                        "_name": f"completion_match__{q}",
                        "path": "completion_terms",
                        "query": {
                            "match": {
                                "completion_terms.tag.edge_ngram": {
                                    "query": q,
                                }
                            }
                        },
                    }
                }
                for q in context
            ]
        }
    }
query

{'bool': {'filter': [{'nested': {'_name': 'completion_match__ABS',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'ABS'}}}}},
   {'nested': {'_name': 'completion_match__Airbags',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'Airbags'}}}}}]}}

In [101]:
query

{'bool': {'filter': [{'nested': {'_name': 'completion_match__ABS',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'ABS'}}}}},
   {'nested': {'_name': 'completion_match__Airbags',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'Airbags'}}}}}]}}

In [102]:
resp = es.es.search(
    index=idx,
    query=query,
    size=0,
    # filter_path="took,**.buckets.key,**.buckets.doc_count",
    #explain=True,
)
resp.body

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2503, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [103]:
query_string

'stability co'

In [104]:
get_completion_prefix(query_string)

'co'

In [105]:
number_of_search_terms_from(get_completion_prefix(query_string))

1

In [106]:
aggs = None

if number_of_search_terms_from(query_string)<=1:
     aggs = {
        "topics_parent": {
            "nested": {"path": "completion_terms"},
            "aggs": {
                "topics": {
                    "filter": {
                        "bool": {
                            "should": [
                                {
                                    "term": {
                                        "completion_terms.tag.edge_ngram": get_completion_prefix(
                                            query_string
                                        )
                                    }
                                },
                                {
                                    "match_phrase": {
                                        "completion_terms.tag.edge_ngram": {
                                            "query": get_completion_prefix(query_string)
                                        }
                                    }
                                },
                                {
                                    "match": {
                                        "completion_terms.tag.edge_ngram": {
                                            "query": get_completion_prefix(
                                                query_string
                                            ),
                                            "operator": "OR",
                                            "fuzziness": "AUTO",
                                            "prefix_length": max(
                                                1,
                                                math.floor(
                                                    (1 / 3)
                                                    * len(
                                                        get_completion_prefix(
                                                            query_string
                                                        )
                                                    )
                                                ),
                                            ),
                                        }
                                    }
                                },
                            ]
                        }
                    },
                    "aggs": {
                        "topics": {
                            "terms": {
                                "field": "completion_terms.tag.keyword",
                                "size": 10,
                            },
                        }
                    },
                }
            },
        }
    }
     
elif number_of_search_terms_from(query_string) == number_of_search_terms_from(get_completion_prefix(query_string)):
      aggs = {
        "topics_parent": {
            "nested": {"path": "completion_terms"},
            "aggs": {
                "topics": {
                    "filter": {
                        "bool": {
                            "should": [
                                {
                                    "multi_match": {
                                        "query": get_completion_prefix(query_string),
                                        "type": "bool_prefix",
                                        "fields": [
                                            "completion_terms.tag",
                                            "completion_terms.tag._2gram",
                                            "completion_terms.tag._3gram",
                                        ],
                                    }
                                }
                            ]
                        }
                    },
                    "aggs": {
                        "topics": {
                            "terms": {
                                "field": "completion_terms.tag.keyword",
                                "size": 10,
                                "include": f"{get_completion_prefix(query_string)}.*",
                            },
                        }
                    },
                }
            },
        }
    }
     
else:
      aggs = {
        "topics_parent": {
            "nested": {"path": "completion_terms"},
            "aggs": {
                "topics": {
                    "filter": {
                        "bool": {
                            "should": [
                                {
                                    "term": {
                                        "completion_terms.tag.edge_ngram": get_completion_prefix(
                                            query_string
                                        )
                                    }
                                },
                                {
                                    "match_phrase": {
                                        "completion_terms.tag.edge_ngram": {
                                            "query": get_completion_prefix(query_string)
                                        }
                                    }
                                },
                                {
                                    "match": {
                                        "completion_terms.tag.edge_ngram": {
                                            "query": get_completion_prefix(
                                                query_string
                                            ),
                                            "operator": "OR",
                                            "fuzziness": "AUTO",
                                            "prefix_length": max(
                                                1,
                                                math.floor(
                                                    (1 / 3)
                                                    * len(
                                                        get_completion_prefix(
                                                            query_string
                                                        )
                                                    )
                                                ),
                                            ),
                                        }
                                    }
                                },
                            ]
                        }
                    },
                    "aggs": {
                        "topics": {
                            "terms": {
                                "field": "completion_terms.tag.keyword",
                                "size": 10,
                                 "include": f"{get_completion_prefix(query_string)}.*",
                            },
                        }
                    },
                }
            },
        }
    }


          


    
     


aggs  

{'topics_parent': {'nested': {'path': 'completion_terms'},
  'aggs': {'topics': {'filter': {'bool': {'should': [{'term': {'completion_terms.tag.edge_ngram': 'co'}},
       {'match_phrase': {'completion_terms.tag.edge_ngram': {'query': 'co'}}},
       {'match': {'completion_terms.tag.edge_ngram': {'query': 'co',
          'operator': 'OR',
          'fuzziness': 'AUTO',
          'prefix_length': 1}}}]}},
    'aggs': {'topics': {'terms': {'field': 'completion_terms.tag.keyword',
       'size': 10,
       'include': 'co.*'}}}}}}}

In [107]:
resp = es.es.search(
    index=idx,
    query=query,
    aggs=aggs,
    size=0,
    source="completion_terms",
    filter_path="took,**.buckets.key,**.buckets.doc_count",
)
resp.body

{'took': 10,
 'aggregations': {'topics_parent': {'topics': {'topics': {'buckets': [{'key': 'coupe',
       'doc_count': 136},
      {'key': 'convertible', 'doc_count': 82},
      {'key': 'coupe, convertible', 'doc_count': 70},
      {'key': 'convertible, coupe', 'doc_count': 27},
      {'key': 'coupe, sedan', 'doc_count': 12},
      {'key': 'corolla', 'doc_count': 11},
      {'key': 'coupe, sedan, convertible', 'doc_count': 10},
      {'key': 'continental', 'doc_count': 9},
      {'key': 'comfortable seating', 'doc_count': 8},
      {'key': 'convertible, sedan, coupe', 'doc_count': 7}]}}}}}

In [108]:
query

{'bool': {'filter': [{'nested': {'_name': 'completion_match__ABS',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'ABS'}}}}},
   {'nested': {'_name': 'completion_match__Airbags',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'Airbags'}}}}}]}}